In [ ]:
from imutils.video import VideoStream
from imutils.video import FileVideoStream
from google.colab.patches import cv2_imshow
import cv2
import imutils
import time

In [ ]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# grab the dimensions of the frame and then construct a blob from it 
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
	faceNet.setInput(blob)
	detections = faceNet.forward()

	# initialize our list of faces, their corresponding locations,
	# and the list of predictions from our face mask network
	faces = []
	locs = []
	preds = []

	# loop over the detections
	for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with the detection 
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
		if confidence > 0.4:
			# compute the (x, y)-coordinates of the bounding box for the object 
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# ensure the bounding boxes fall within the dimensions of the frame 
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# extract the face ROI, convert it from BGR to RGB channel
			# ordering, resize it to img_sizeximg_size, and preprocess it
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (img_size, img_size))
			face = img_to_array(face)
			face = preprocess_input(face)

			# add the face and bounding boxes to their respective lists 
			faces.append(face)
			locs.append((startX, startY, endX, endY))

	# only make a predictions if at least one face was detected
	if len(faces) > 0:
		# for faster inference we'll make batch predictions on *all* faces at the same
		# time rather than one-by-one predictions in the above `for` loop 
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	# return a 2-tuple of the face locations and their corresponding locations 
	return (locs, preds)

In [ ]:
# load our serialized face detector model
faceNet = cv2.dnn.readNet('/content/deploy.prototxt', '/content/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
# prediction model
maskNet = model

In [ ]:
# initialize the video stream
print("[INFO] starting video stream...")
vs = FileVideoStream('/content/dedo.mp4').start()

In [ ]:
# video writer & output path
writer = None
VIDEO_STREAM_OUT = '/content/video_dedo.avi'

In [ ]:
# loop over the frames from the video stream
while True:
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
	frame = vs.read()
	try:
		frame = imutils.resize(frame, width=400)
	except Exception as ex:
		print(ex)
	
	# detect faces in the frame and determine if they are wearing a
	# face mask or not
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# loop over the detected face locations and their corresponding
	# locations
	for (box, pred) in zip(locs, preds):
		# unpack the bounding box and predictions
		(startX, startY, endX, endY) = box
		(old_with_mask, old_without_mask, young_with_mask, young_without_mask) = pred

		# determine the class label and color we'll use to draw
		# the bounding box and text
		def label_prediction(old_with_mask, old_without_mask, young_with_mask, young_without_mask):
			if old_with_mask > old_without_mask and old_with_mask > young_with_mask and old_with_mask > young_without_mask:
				return str("old_with_mask")
			elif old_without_mask > old_with_mask and old_without_mask > young_with_mask and old_without_mask > young_without_mask:
				return str("old_without_mask")
			elif young_with_mask > old_with_mask and young_with_mask > old_without_mask and young_with_mask > young_without_mask:
				return str("young_with_mask")
			elif young_without_mask > old_with_mask and young_without_mask > old_without_mask and young_without_mask > young_with_mask:
				return str("young_without_mask")

		label = label_prediction(old_with_mask, old_without_mask, young_with_mask,young_without_mask)
		
		def color_prediction(old_with_mask, old_without_mask, young_with_mask, young_without_mask):
			if old_with_mask > old_without_mask and old_with_mask > young_with_mask and old_with_mask > young_without_mask:
				return (0, 255, 0)
			elif old_without_mask > old_with_mask and old_without_mask > young_with_mask and old_without_mask > young_without_mask:
				return (0, 0, 255)
			elif young_with_mask > old_with_mask and young_with_mask > old_without_mask and young_with_mask > young_without_mask:
				return (70, 219, 224)
			elif young_without_mask > old_with_mask and young_without_mask > old_without_mask and young_without_mask > young_with_mask:
				return (201, 26, 166)			
		
		color = color_prediction(old_with_mask, old_without_mask, young_with_mask, young_without_mask)

		# include the probability in the label
		label = "{}: {:.2f}%".format(label, max(old_with_mask, old_without_mask, young_with_mask, young_without_mask) * 100)

		# display the label and bounding box rectangle on the output
		# frame
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# show the output frame
	cv2_imshow(frame)
	key = cv2.waitKey(1) & 0xFF

	# Check if the video writer is None
	if writer is None:
		# Initialize our video writer
		fourcc = cv2.VideoWriter_fourcc(*"XVID")
		writer = cv2.VideoWriter(VIDEO_STREAM_OUT, fourcc, 30, (frame.shape[1], frame.shape[0]), True) 
	
	# Write the output video
	writer.write(frame)
	
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# Release the file pointers
print("[INFO] cleaning up...")
writer.release()

# cleanup
cv2.destroyAllWindows()
vs.stop()